In [117]:
# Select the review that bests represents a given topic
# Select the business with the highest proportion of a given topic

import cPickle as pickle
from gensim import corpora, models, similarities, utils
import numpy as np
import pandas as pd
from KAsql import query_SQL
import pyLDAvis.gensim
import pyLDAvis

In [118]:
#lda = pickle.load(open('lda.p','rb'))
#lda.print_topics()

In [124]:
# Load dfs with gammas for all reviews and averaged by business/doctor
# All reviews
ids_gammas_df = pickle.load(open('ids_gammas_df.p', 'rb'))#actually np array until I rerun NLP_all_reviews-LDA
ids_gammas_df.head(3)

# Avg gamma for each doctor/business
bid_gmeans = pickle.load(open('bid_gmeans.p', 'rb'))
bid_gmeans.head(3)

,1,2,3,4,5,6,7,8,9,10
BID,,,,,,,,,,
189,0.026678,0.222100,0.016266,0.120362,0.126442,0.149261,0.026810,0.051409,0.113959,0.146712
190,0.060923,0.143381,0.022829,0.057863,0.090280,0.131948,0.098921,0.175391,0.049655,0.168808
191,0.041454,0.253812,0.043703,0.061581,0.110030,0.054151,0.136972,0.154059,0.064457,0.079781


In [140]:
def three_representative_reviews(mytopic,ids_gammas_df):
    """What 3 reviews best represent a given topic?
    Those with the highest gamma-weights on one topic
    Revise to print top 3 reviews and see if they are consistent and validate with stars
    - use these reviews to help define & validate each topic"""

    #ids_gammas_df.head()#Check what df looks like prior to sorting

    # Sort on a given topic
    mysorted = ids_gammas_df.sort(mytopic, axis=0, ascending=False)
    idx = mysorted.index[0:3]# get index of top-ranking review for mytopic

    RID = mysorted.loc[idx]['RID']#Note that if you replace idx with 0, and print the results, it is not the same
    RIDs = [str(int(value)) for value in RID.values]#list of strings (use int to get rid of decimal)

    #For debugging
    #print 'Topic: ',mytopic,'\n'
    #print 'Index: ',idx
    print 'RID: ',RID.values,'\n'#type Series
    
    # Pull 3 reviews, and see if they appear to represent one topic
    #from KAsql import query_SQL

    # SQL query uses my function
    #from KAsql import query_SQL
    sql = 'SELECT id, stars, comment FROM review WHERE id='+RIDs[0]+' or id='+RIDs[1]+' or id='+RIDs[2]+';'
    rows = query_SQL(sql)

    sqlRID=[];stars=[];comment=''
    sqlRID = rows[:][0]
    stars = rows[:][1]
    comment = rows[:][2]

    # Print Statements
    print sql,'\n'
    print '\nTop three reviews most related to topic: ',mytopic,'\n'
    print mysorted.head(3),'\n'#Confirm RIDs match with sql data

    # For some reason rows are printed in opposite order from listing
    for row in rows:
        print row[0],row[1],row[2],'\n'

In [144]:
three_representative_reviews(2,ids_gammas_df)

RID:  [ 4263.  3964.  4058.] 

SELECT id, stars, comment FROM review WHERE id=4263 or id=3964 or id=4058; 


Top three reviews most related to topic:  2 

       RID  BID         1         2         3         4         5         6  \
3594  4263  311  0.000224  0.997986  0.000224  0.000224  0.000224  0.000224   
3295  3964  302  0.000230  0.997931  0.000230  0.000230  0.000230  0.000230   
3389  4058  304  0.000246  0.997788  0.000246  0.000246  0.000246  0.000246   

             7         8         9        10  
3594  0.000224  0.000224  0.000224  0.000224  
3295  0.000230  0.000230  0.000230  0.000230  
3389  0.000246  0.000246  0.000246  0.000246   

3964 2.0 I'm writing this review based on my personal experience at FDFAC and with Dr. Sanders. I am happy for those who have had good experiences at this center; however, I can't second that. My experience in terms of dealing with the staff at FDFAC and with Dr. Sanders directly was far from bad and in fact I will commend them for bein

In [142]:
def most_representative_review(mytopic,ids_gammas_df):
    """ What review best represents a given topic?
    Selects the review with the highest gamma-weights on one topic.
    Accepts mytopic (an int corresponding to topic #) and the Rev x G matrix with rid, bid and gammas"""

    #print(ids_gammas_df.head())#Check what df looks like prior to sorting

    # Sort on a given topic
    mysorted = ids_gammas_df.sort(mytopic, axis=0, ascending=False)
    idx = mysorted.index[0]# get index of top-ranking review for mytopic

    RID = int(mysorted.loc[idx]['RID'])
    print 'Index = ',idx, ' Starting RID: ',RID,' Topic: ',mytopic,'\n'#Note that if you replace idx with 0, and print the results, it is not the same
    #print(mysorted.loc[0]['RID'])# does not work

    #from KAsql import query_SQL
    sql = 'SELECT id, stars, comment FROM review WHERE id='+str(RID)+';'
    rows = query_SQL(sql)

    sqlid = rows[0][0]
    stars = rows[0][1]
    comment = rows[0][2]
    
    #Print statements
    print sql,'\n'
    print '\nTop review most related to topic: ',mytopic,'\n'
    print(mysorted.head(3))
    print str(RID), sqlid, stars, comment

In [143]:
most_representative_review(3,ids_gammas_df)

Index =  3410  Starting RID:  4079  Topic:  3 

SELECT id, stars, comment FROM review WHERE id=4079; 


Top review most related to topic:  3 

       RID  BID         1         2         3         4         5         6  \
3410  4079  305  0.000565  0.000565  0.994914  0.000565  0.000565  0.000565   
3190  3859  298  0.000641  0.000641  0.994229  0.000641  0.000641  0.000641   
102    771  191  0.000769  0.000769  0.993075  0.000769  0.000769  0.000769   

             7         8         9        10  
3410  0.000565  0.000565  0.000565  0.000565  
3190  0.000641  0.000641  0.000641  0.000641  
102   0.000769  0.000769  0.000769  0.000769  
4079 4079 5.0 If I could give more stars, then I would. I met Dr. Sloan on a Sunday on his day off! in the lobby of a hospital because I was stricken by shingles in the ear over the weekend that led to facial paralysis on the leftside of my face  a rare condition called Ramsay Hunt. Dr. Sloan is the best combination of a personal caring doctor with s